In [1]:
import clingo
from gamearg import *
from IPython.display import display, Image
import pandas as pd
import concurrent
from concurrent.futures import ThreadPoolExecutor
import time

## Generating Graph 

With given `node_num`, program will be able to generate all possible edges

In [2]:
def graph_generating(node_num):
    def on_model(m):
        if str(m) == "":
            possible_graphs.append(str(m))
        else:
            elements = str(m).split(" ")
            elements_with_dot = [element + "." for element in elements]
            concatenated = " ".join(elements_with_dot)
            possible_graphs.append(concatenated)

    graph_log = """
    #const n = {}.
    node(1..n).
    {{move(X,Y): node(X), node(Y),X!=Y}}.
    #show move/2.
    """.format(node_num)
    # print(graph_log)
    ctl = clingo.Control()
    ctl.configuration.solve.models = "0"
    ctl.add("base", [], graph_log)
    ctl.ground([("base", [])])
    possible_graphs = []
    ctl.solve(on_model=on_model)
    return possible_graphs

In [3]:
possible_graphs = graph_generating(node_num=2)

In [4]:
possible_graphs

['', 'move(2,1).', 'move(1,2).', 'move(1,2). move(2,1).']

## Digest into Game Solver

Based on the `possible_graphs`, we need to digest them into game_solver one by one and ensure that the result will be 3-valued (dir=back and forward)

In [5]:
def nodes_connected(graph):
    graph_log = f"""
    {graph}
    move(X,Y):- move(Y,X).
    node(X):- move(X,_).
    node(X):- move(_,X).
    connected(X,Y) :- move(X,Y).
    connected(X,Y) :- connected(X,Z), move(Z,Y).
    :- node(X), node(Y), not connected(X,Y).
    """
    ctl = clingo.Control()
    ctl.configuration.solve.models = "0"
    ctl.add("base", [], graph_log)
    ctl.ground([("base", [])])
    return(ctl.solve())

In [6]:
columns = ["node_number", "edge_number", "graph", "fw_node_wfs", "bw_node_wfs", "connected"]
graph_cal = pd.DataFrame(columns=columns)
node_num = 1

def process_graph(graph, node_num):
    try:
        fw_node_wfs_df = node_wfs_cal(graph, "game", reverse=False)
        bw_node_wfs_df = node_wfs_cal(graph, "game", reverse=True)
        return {
            "node_number": node_num,
            "edge_number": len(graph.split(".")) - 1,
            "graph": graph,
            "fw_node_wfs": set(fw_node_wfs_df["wfs"]),
            "bw_node_wfs": set(bw_node_wfs_df["wfs"]),
            "connected": str(nodes_connected(graph))
        }
    except Exception as e:
        print(f"An error occurred while processing {graph}: {e}")
        return None

data_to_add = []
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = []
    for node_num in range(1, 6):
        possible_graphs = graph_generating(node_num)
        if not possible_graphs or all(graph == "" for graph in possible_graphs):
            continue
        for graph in possible_graphs:
            if graph == "":
                continue
            else:
                futures.append(executor.submit(process_graph, graph, node_num))
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result:
            data_to_add.append(result)
graph_cal = pd.DataFrame(data_to_add, columns=columns)

In [7]:
graph_cal

,node_number,edge_number,graph,fw_node_wfs,bw_node_wfs,connected
0,3,3,"move(1,3). move(2,1). move(2,3).","{won, lost}","{won, lost}",SAT
1,4,5,"move(1,2). move(2,1). move(3,1). move(3,4). mo...",{draw},"{won, lost}",SAT
2,3,2,"move(1,3). move(2,3).","{won, lost}","{won, lost}",SAT
3,4,7,"move(1,2). move(1,3). move(1,4). move(2,3). mo...",{draw},{draw},SAT
4,4,9,"move(1,2). move(1,3). move(1,4). move(2,1). mo...",{draw},{draw},SAT
...,...,...,...,...,...,...
1052731,5,19,"move(1,2). move(1,3). move(1,4). move(1,5). mo...",{draw},{draw},SAT
1052732,5,18,"move(1,2). move(1,4). move(1,5). move(2,1). mo...",{draw},{draw},SAT
1052733,5,19,"move(1,2). move(1,4). move(1,5). move(2,1). mo...",{draw},{draw},SAT
1052734,5,19,"move(1,2). move(1,3). move(1,4). move(1,5). mo...",{draw},{draw},SAT


In [8]:
graph_cal['fw_len'] = graph_cal['fw_node_wfs'].apply(len)
graph_cal['bw_len'] = graph_cal['bw_node_wfs'].apply(len)

In [9]:
graph_cal[(graph_cal["fw_len"] == 3) & (graph_cal["bw_len"] == 3)].sort_values(["node_number", "edge_number"])

,node_number,edge_number,graph,fw_node_wfs,bw_node_wfs,connected,fw_len,bw_len
1185,4,3,"move(2,1). move(3,4). move(4,3).","{won, draw, lost}","{won, draw, lost}",UNSAT,3,3
1241,4,3,"move(1,4). move(2,3). move(3,2).","{won, draw, lost}","{won, draw, lost}",UNSAT,3,3
1357,4,3,"move(1,2). move(2,1). move(4,3).","{won, draw, lost}","{won, draw, lost}",UNSAT,3,3
1381,4,3,"move(1,2). move(2,1). move(3,4).","{won, draw, lost}","{won, draw, lost}",UNSAT,3,3
1570,4,3,"move(2,4). move(3,1). move(4,2).","{won, draw, lost}","{won, draw, lost}",UNSAT,3,3
...,...,...,...,...,...,...,...,...
990466,5,9,"move(1,2). move(1,5). move(2,1). move(2,5). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3
990552,5,9,"move(1,2). move(1,5). move(2,1). move(2,5). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3
998721,5,9,"move(1,2). move(1,3). move(1,4). move(2,1). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3
999562,5,9,"move(1,2). move(1,4). move(2,1). move(2,4). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3


In [10]:
graph_cal[(graph_cal["fw_len"] == 3) & (graph_cal["bw_len"] == 3) & (graph_cal["connected"] == "SAT")].sort_values(["node_number", "edge_number"])

,node_number,edge_number,graph,fw_node_wfs,bw_node_wfs,connected,fw_len,bw_len
6224,5,5,"move(1,3). move(1,4). move(2,3). move(3,5). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3
6297,5,5,"move(1,3). move(1,4). move(2,5). move(3,5). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3
6406,5,5,"move(1,4). move(2,3). move(2,5). move(3,4). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3
6446,5,5,"move(1,4). move(2,3). move(3,4). move(3,5). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3
6492,5,5,"move(1,4). move(2,3). move(3,1). move(3,5). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3
...,...,...,...,...,...,...,...,...
990466,5,9,"move(1,2). move(1,5). move(2,1). move(2,5). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3
990552,5,9,"move(1,2). move(1,5). move(2,1). move(2,5). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3
998721,5,9,"move(1,2). move(1,3). move(1,4). move(2,1). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3
999562,5,9,"move(1,2). move(1,4). move(2,1). move(2,4). mo...","{won, draw, lost}","{won, draw, lost}",SAT,3,3


In [11]:
graph_cal.to_csv("graph_cal_non_selfloop.csv")